In [0]:
# !pip install typing-extensions
# !pip install openai==1.38.0

# dbutils.library.restartPython()

In [0]:
import os
import pandas as pd
import ast
import re
import openai
import pprint
import random
import pyspark.sql.functions as F
import numpy as np
from openai import OpenAI

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

In [0]:
filename_data = ""
data = pd.read_csv(filename_data)[6000::]
data = data.drop(columns="row_data")
data.display()
print(len(data))

In [0]:
filename_annotation_data = ""
annotation_data = pd.read_json(filename_annotation_data, lines=True)
annotation_data.to_json(filename_annotation_data, lines=True, orient="records")

In [0]:
annotation_data.display()

In [0]:
def get_clean_labels(labels):
  """
  Extracts the clean labels from the exported label structure
  """
  label_processed = []

  PROJECT_ID = "cm1q6nik6098e08xp0d9n176x"
  for i, row in labels.iterrows():
    for label in row['projects'][PROJECT_ID]['labels']: 
        c = label["annotations"]['classifications']
        c_df = {}

        c_df["Global Key"]  = int(row['data_row']['global_key'])
        c_df["Labeler"] = label["label_details"]['created_by']
        c_df["Description"] = row['data_row']["row_data"]
  
        if c != []:
          c_df["Annotation"] = label["annotations"]['classifications'][0]['radio_answer']['value']
        else:
            try:
              c_df["Annotation"] = label["annotations"]['objects'][0]['classifications'][0]['checklist_answers'][0]['value']
              c_df["Annotation_start"] = str(label["annotations"]['objects'][0]['location']['start'])
              c_df["Annotation_end"] = str(label["annotations"]['objects'][0]['location']['end'])
              c_df["Annotation_token"] = str(label["annotations"]['objects'][0]['location']['token'])
              c_df["Category"] = label["annotations"]['objects'][0]['value']
            except:
              pass

        c_df = pd.DataFrame.from_dict([c_df])
        label_processed += [c_df]

  processed = pd.concat(label_processed)

  return processed

In [0]:
# Specifies email of the annotator that has to be removed
email = ""

annotation_data_clean = get_clean_labels(annotation_data)
annotation_data_clean['INCLUSIEF'] = annotation_data_clean['Annotation'] == "er_is_hier_geen_sprake_van_niet_inclusief_taalgebruik"
annotation_data_clean = annotation_data_clean[annotation_data_clean['Labeler'] != email]
annotation_data_clean = annotation_data_clean[annotation_data_clean['Global Key'] > 6000]
annotation_data_clean = annotation_data_clean.sort_values(by='Global Key')
annotation_data_clean = annotation_data_clean[annotation_data_clean['Annotation'].notna()]
annotation_data_clean.display()

In [0]:
annotation_df = annotation_data_clean.groupby(['Global Key']).agg({
    'Global Key': 'first',
    'INCLUSIEF': 'all',
    'Annotation': lambda x: ' | '.join(str(i) for i in x),
    'Category' : lambda x: ' | '.join(str(i) for i in x)
})
annotation_df.display()

In [0]:
count = 0

for i in annotation_df['INCLUSIEF']:
    if i:
        count += 1

print("Number of non inclusive datarows:", len(annotation_df) - count)
print("Dataset length:",len(annotation_data_clean))


In [0]:
gender = [
    {"""Zij is de hoofdredactrice in het project."""},
    
    {"""De 47-jarige Sietsma werkt sinds 2017 als onderzoeksjournaliste voor het NOS/NTR-programma Nieuwsuur, en was daarvoor al zeven jaar werkzaam voor RTL Nieuws, ook als journaliste bij de onderzoeksredactie. Eerdere werkervaring deed hij op bij de actualiteitenprogramma’s Twee Vandaag en Netwerk. """},

    {"""De vrouwelijke CEO gaf een speech."""},

    {"""Premier Rutte en vicepremier Sigrid Kaag"""}
]

seksuele_orientatie = [
    {"""Er wordt vaak neergekeken op mensen met een andere seksualiteit"""}, 
                        
    {"""De groep bestaat met name uit homo's"""}, 

    {"""Een panseksueel is iemand die zich aangetrokken voelt tot alle genderidentiteiten."""}
]

etniciteit = [
    {"""Veel blanke mensen vinden het moeilijk om zich bewust te zijn van hun privilege"""}, 

    {"""Dit verhaal zal veel mensen met een andere huidskleur bekend voorkomen."""}, 

    {"""De cast bestaat met name uit allochtonen"""}
]

beperking = [
    {"""Gehandicapte mensen zijn vaak de dupe."""}, 

    {"""Dit programma is toegankelijk voor doven en slechthorenden"""}, 

    {"""De hoofdpersoon lijdt aan autisme"""},

    {"""Bas is blind, maar zit toch in de jury"""}
]

inclusief = [
    {"""De documentaire vertelt het verhaal van praktisch opgeleide vrouwen."""}, 
    
    {"""Dit programma is toegangelijk voor dove en slechthorende mensen"""},

    {"""De groep bestaat met name uit homo mannen"""}
]

In [0]:
def initialize_openai_api(): 
  """
  Initializes the OpenAI api.
  """
  vault, key = "", ""
  endpoint = ""
  os.environ['OPENAI_API_KEY'] = dbutils.secrets.get(vault, key)
  openai.api_type = "azure"
  openai.azure_endpoint = endpoint
  openai.api_version = "2024-02-01"
  openai.api_key = os.getenv("OPENAI_API_KEY")

def get_initial_prompt_general():
  """
  Returns the role and content for the prompt. The prompts exists of multiple different examples.
  """
  gender1 = gender[random.randint(0, len(gender) - 1)]
  seksuele_orientatie1 = seksuele_orientatie[random.randint(0, len(seksuele_orientatie)-1)]
  etniciteit1 = etniciteit[random.randint(0, len(etniciteit)-1)]
  beperiking1 = beperking[random.randint(0, len(beperking) - 1)]
  inclusief1 = inclusief[random.randint(0, len(inclusief) - 1)]

  initial_prompt = f"""
    Je bent een systeem dat gespecializeerd is in het detecteren van niet-inclusief taalgebruik in artikelen van RTL. Je doel is om te detecteren of er sprake is van niet-inclusief taalgebruik. Dit zijn criteria voor niet-inclusief taalgebruik: 
    
1. Gender
- Vrouwelijke vorm van beroepsnamen (Bijv. hoofdredactrice, voetbalsters)
- Genderspecifieke beroepsnamen. (Bijv. politiemannen)
- Onnodige bijvoegelijke naamwoorden (Bijv. de vrouwelijke CEO)
- Inconsistente achternamen (Bijv. Sigrid Kaag met achternaam en Rutte zonder)
- Onjuiste voornaamwoorden (Als gender bekend is, gebruik de juiste voornaamwoorden zij/haar, hij/hem, die/diens etc.)
- Stigmatiseren zoals gebruik van "mannen en vrouwen" ipv mensen (Bijv. meisje ipv vrouw, 300 man ipv 300 mensen)
- Incorrect gebruik van terminologie (Bijv. een transman ipv trans man, een non-binair ipv non-binair persoon)

2. Seksuele orientatie
- Othering (Bijv. mensen met een andere seksuele orientatie ipv lhbti-personen)
- Seksualiteit als bijvoegelijk naamwoord (Bijv. homo's ipv homo mannen)
- Stigmatiseren (Bijv. gebruik van woorden zoals homo's, flikkers etc.)
- Incorrect gebruik van terminologie (Bijv. een homo ipv een homo man, een panseksueel ipv pansexuele mensen)

3. Etniciteit & culturele achtergrond
- Verkeerd gebruik van zwart en wit (Bijv. blanke mensen ipv zwarte en witte mensen/mensen van kleur)
- Onnodig gebruik van etniciteit (alleen als het relevant is voor de context)
- Othering (Bijv. een andere huidskleur)
- Etniciteit als bijvoegelijk naamwoord (Bijv. zwarten ipv zwarte mensen)
- n-woord (dit woord mag alleen gebruikt worden in historiche context wanneer uitleg en duiding mogelijk is)
- Incorrect gebruik van terminologie (Bijv. gebruik van allochtonen of niet-witte mensen) 

4. Fysieke/mentale beperking
- Gebruik van minder valide/invalide/handicap ipv mensen met een beperking
- Onnodige bijvoegelijke naamwoorden (Bijv. doven en slechthorenden ipv dove en slechthorende mensen)
- Gebruik van "lijden aan" (Bijv. "lijdt aan autisme" ipv "heeft autisme")
- Gebruik van constructies waar een beperking een uitzondering is (Bijv. hij is blind, maar zit toch in de jury)
- Onnodige bijvoegelijke naamwoorden (Bijv. blinden ipv blinde mensen)
- Stigmatiseren (Bijv. dat was een beetje autistisch)
- Incorrect gebruik van terminologie (Bijv. downies ipv mensen met het syndroom van Down)

5. Leeftijd
- Gebruik van "bejaarden" of "oudjes" in plaats van "ouderen" of 60-plussers
- Inconsistencie van leeftijd (Bijv. alleen leeftijd van de oudste vermelden)

6. Opleiding
- Hiërarchie tussen mensen van verschillende opleidingsrichtingen (Bijv. laagopgeleid ipv mbo-, hbo- of wo-opgeleid of praktisch en theoretisch opgeleid) 
- Onderscheid maken tussen studenten/scholieren/leerlingen. Mbo, hbo en wo vallen allemaal onder de noemer student

7. Overig
- Als een paragraaf duidelijk niet-inclusief taalgebruik bevat maar het niet onder bovenstaande categorien behoort, label het ook als niet-inclusief.

Beoordeel de tekst systematisch per categorie: Gender, Seksuele oriëntatie, Etniciteit, Fysieke/Mentale beperking, Leeftijd, Opleiding, Overig. Gebruik de criteria als checklist. Je krijgt 5 paragrafen tegelijk te zien. Je reactie bevat voor al die paragrafen een classificatie van welke van de 7 categorieën er sprake is. Geef naast de categorie ook een reden voor de classificatie zodat een schrijver begrijpt hoe je tot die conclusie bent gekomen. De paragraaf die beoordeeld moet worden staat tussen <start-of-text> en <end-of-text>. 

Stappen:
1. Identificeer de relevante woorden/zinnen in de paragraaf
2. Classificeer of deze niet-inclusief zijn
3. Leg uit waarom.
4. Double-check de categorie en reden
5. Zorg voor een consistente en nauwkeurige JSON-output zoals die als voorbeeld is gegeven. 
6. Check nogmaals of er een vrouwelijke vorm van beroepsnamen voorkomt in de tekst, zo ja: label deze als niet-inclusief gebruik van gender.

Een output is succesvol als het taalgebruik correct is geclassificeerd volgens de opgegeven definities, met duidelijke uitleg en een antwoord in correct JSON-formaat.

Voorbeeld:

1. <start-of-text> {etniciteit1} <end-of-text>
2. <start-of-text> {beperiking1} <end-of-text>
3. <start-of-text> {seksuele_orientatie1} <end-of-text>
4. <start-of-text> {gender1} <end-of-text>
5. <start-of-text> {inclusief1} <end-of-text>

Output:

{{1: {{"Gender": "0", "Seksuele orientatie": "0", "Etniciteit & culturele achtergrond": "1", "Fysieke/mentale beperking": "0", "Leeftijd": "0", "Opleiding": "0", "Overig": "0", "REDEN": "..."}},
2: {{"Gender": "0", "Seksuele orientatie": "0", "Etniciteit & culturele achtergrond": "0", "Fysieke/mentale beperking": "1", "Leeftijd": "0", "Opleiding": "0", "Overig": "0", "REDEN": "..."}},
3: {{"Gender": "0", "Seksuele orientatie": "1", "Etniciteit & culturele achtergrond": "0", "Fysieke/mentale beperking": "0", "Leeftijd": "0", "Opleiding": "0", "Overig": "0", "REDEN": "..."}},
4: {{"Gender": "1", "Seksuele orientatie": "0", "Etniciteit & culturele achtergrond": "0", "Fysieke/mentale beperking": "0", "Leeftijd": "0", "Opleiding": "0", "Overig": "0", "REDEN": "..."}},
5: {{"Gender": "0", "Seksuele orientatie": "0", "Etniciteit & culturele achtergrond": "0", "Fysieke/mentale beperking": "0", "Leeftijd": "0", "Opleiding": "0", "Overig": "0", "REDEN": "Geen sprake van niet-inclusief taalgebruik"}}}}

----------------------------------------------------------------------------------

Bedankt voor het helpen bij het zorgen voor een inclusieve omgeving!
"""
    return [{"role": "system", "content": initial_prompt}]

def append_paragraph(messages, paragraphs):
  """
  Appends five to-be-labeled datapoints to the prompt.
  """
  messages[0]['content'] += (f"""1. <start-of-text> {paragraphs[0]} <end-of-text>
                   
                   2. <start-of-text> {paragraphs[1]} <end-of-text>
                   
                   3. <start-of-text> {paragraphs[2]} <end-of-text>
                   
                   4. <start-of-text> {paragraphs[3]} <end-of-text>
                   
                   5. <start-of-text> {paragraphs[4]} <end-of-text>""")
  return messages

def rate_paragraph(paragraphs):
  """
  This function calls to the OpenAI API to label the given data points. If the result is not able to be converted to a data structure, it returns None.
  """
  prompts = append_paragraph(get_initial_prompt_general(), paragraphs)[0]
  response = openai.chat.completions.create(
                                        model="gpt-4o",
                                        messages=[
                                          {
                                            "role": prompts['role'],
                                            "content": prompts['content'],
                                            "type": "json"
                                          }
                                        ],
                                        temperature=0.5 # 1
                                        )
  try: 
    score_dict = ast.literal_eval(response.choices[0].message.content)
    return score_dict
  except Exception as e:
    try:
      score_dict = ast.literal_eval(response.choices[0].message.content.strip("`json"))
      return score_dict
    except:
      print(e)
      return None

def label_sample(to_label, int_id):
  """
  This function loops through all the data points and calls the OpenAI API to label them in batches of five. If a None is returned it tries again up to five times. If it still fails, it sets all values to None sets the amount of nones to five. It also prints the progress every batch of 200.
  """
  nones = 0
  dfs = []
  shape = to_label.shape[0]
  for i in range(0,shape,5):
    if i%200 == 0:
        print(f"{str(i)} of {str(shape)} done, of which {str(nones)} nones")
    sample = to_label.iloc[i:min(i+5, shape)].copy()
    for j in range(5):
      try:
        output = rate_paragraph(sample['paragraphs'].values)
        result_df = pd.DataFrame.from_dict(output, orient='index')
        sample[f'Gender{str(int_id)}'] = result_df['Gender'].values
        sample[f'Seksuele orientatie{str(int_id)}'] = result_df['Seksuele orientatie'].values
        sample[f'Etniciteit & culturele achtergrond{str(int_id)}'] = result_df['Etniciteit & culturele achtergrond'].values
        sample[f'Fysieke/mentale beperking{str(int_id)}'] = result_df['Fysieke/mentale beperking'].values
        sample[f'Leeftijd{str(int_id)}'] = result_df['Leeftijd'].values
        sample[f'Opleiding{str(int_id)}'] = result_df['Opleiding'].values
        sample[f'Overig{str(int_id)}'] = result_df['Overig'].values
        sample[f'REDEN{str(int_id)}'] = result_df['REDEN'].values
        break
      except Exception as e:
        output = None
        sample[f'Gender{str(int_id)}'] = None
        sample[f'Seksuele orientatie{str(int_id)}'] = None
        sample[f'Etniciteit & culturele achtergrond{str(int_id)}'] = None
        sample[f'Fysieke/mentale beperking{str(int_id)}'] = None
        sample[f'Leeftijd{str(int_id)}'] = None
        sample[f'Opleiding{str(int_id)}'] = None
        sample[f'Overig{str(int_id)}'] = None
        sample[f'REDEN{str(int_id)}'] = None
    if output == None:
      nones += 5
    dfs += [sample]
  to_label = pd.concat(dfs)
  print(f"{str(shape)} done, of which {str(nones)} nones")
  return to_label

initialize_openai_api()

In [0]:
def label_and_save(index):
  """
  To simulate multiple annotators, all samples will be labeled three times. Then the function will check if the file already exists. If it does, it will return a message that the file already exists. If it doesn't, it will label the data and save it in csv format with the defined filename.
  """
  fname = ""
  if not os.path.isfile(fname):
    to_label = data
    to_label = label_sample(to_label, 1)
    to_label = label_sample(to_label, 2)
    to_label = label_sample(to_label, 3)
    to_label.to_csv(fname,sep=';')
    return f"{str(index)} successfull"
  else:
    return f"{str(index)} existed already"

In [0]:
label_and_save(1)
print("-"*75)
print("DONE")

In [0]:
filename_gpt_data = ""
gpt_data = pd.read_csv(filename_gpt_data, delimiter=';')


In [0]:
gpt_data['INCLUSIEF'] = (
    (gpt_data['Gender1'] == 0) & (gpt_data['Gender2'] == 0) & (gpt_data['Gender3'] == 0) &
    (gpt_data['Seksuele orientatie1'] == 0) & (gpt_data['Seksuele orientatie2'] == 0) & (gpt_data['Seksuele orientatie3'] == 0) &
    (gpt_data['Etniciteit & culturele achtergrond1'] == 0) & (gpt_data['Etniciteit & culturele achtergrond2'] == 0) & (gpt_data['Etniciteit & culturele achtergrond3'] == 0) &
    (gpt_data['Fysieke/mentale beperking1'] == 0) & (gpt_data['Fysieke/mentale beperking2'] == 0) & (gpt_data['Fysieke/mentale beperking3'] == 0) &
    (gpt_data['Leeftijd1'] == 0) & (gpt_data['Leeftijd2'] == 0) & (gpt_data['Leeftijd3'] == 0) &
    (gpt_data['Opleiding1'] == 0) & (gpt_data['Opleiding2'] == 0) & (gpt_data['Opleiding3'] == 0) &
    (gpt_data['Overig1'] == 0) & (gpt_data['Overig2'] == 0) & (gpt_data['Overig3'] == 0)
)

gpt_data.display()

In [0]:
print(len(annotation_data))
len(annotation_df)

In [0]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

gpt_data = gpt_data[gpt_data['global_key'].isin(annotation_df['Global Key'])]

aqua_to_red = LinearSegmentedColormap.from_list("AquaToRed", ["#42625c", "#a1cec5"])
cm = confusion_matrix(annotation_df['INCLUSIEF'], gpt_data['INCLUSIEF'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=aqua_to_red, text_kw={'color': 'white'})
plt.tick_params(axis=u'both', which=u'both',length=0)
plt.grid(False)